<div style="background-color:#EAF4EC; padding:20px; border-radius:10px;">

<h2 style="color:#2F6F4E; text-align:center; margin-bottom:5px;">
Forecasting — Dataset & Baselines
</h2>

<h4 style="color:#2F6F4E; text-align:center; margin-top:0;">
Master Thesis – ESG Governance Indicators (EU-27)
</h4>

<p style="font-size:14px; color:#2F6F4E;">
This notebook implements the <strong>Forecasting</strong> stage of the CRISP-ML(Q) methodology.
The objective is to define a rigorous forecasting setup and build <strong>baseline models</strong> for the
EU-27 governance indicators, creating a benchmark for more advanced machine learning approaches.
</p>

<p style="font-size:14px; color:#2F6F4E;">
This notebook focuses on <strong>problem formulation and benchmarking</strong>. No advanced models are trained here.
The outputs generated will serve as the foundation for forecasting model development and evaluation in subsequent notebooks.
</p>

</div>


In [20]:
import pandas as pd
import numpy as np
from pathlib import Path

In [21]:
DATA_PATH = Path("../data/processed/data_scaled_governance_EU27.csv")
df_scaled = pd.read_csv(DATA_PATH)
print(df_scaled.shape)
df_scaled.head()

(391, 29)


Country Name Country Code                                    Indicator Name  \
0      Austria          AUT                   Control of Corruption: Estimate   
1      Austria          AUT                             GDP growth (annual %)   
2      Austria          AUT                Government Effectiveness: Estimate   
3      Austria          AUT  Individuals using the Internet (% of population)   
4      Austria          AUT                                     Net migration   

      Indicator Code      2000      2001      2002      2003      2004  \
0             CC.EST  0.986735  0.986735  1.195920  1.258012  1.338009   
1  NY.GDP.MKTP.KD.ZG  0.177089 -0.303646 -0.260674 -0.348682  0.016821   
2             GE.EST  1.283242  1.283242  1.337701  1.420691  1.277469   
3     IT.NET.USER.ZS -1.284943 -1.061060 -1.166896 -0.918589 -0.446399   
4        SM.POP.NETM -0.197859 -0.027352 -0.045434  0.007125  0.088755   

       2005  ...      2015      2016      2017      2018      2019      2020  \
0  1.192433  ...  0.631065  0.662725  0.667747  0.754132  0.693885  0.638287   
1 -0.046049  ... -0.307103 -0.098203 -0.058402 -0.003982 -0.191201 -2.263838   
2  1.006125  ...  0.628536  0.667908  0.656246  0.637024  0.696515  0.884421   
3 -0.295786  ...  0.758501  0.774783  0.921325  0.905043  0.917255  0.905043   
4  0.055543  ...  0.550147  0.172870  0.015683 -0.062396 -0.017966 -0.029384   

       2021      2022      2023  missing_pct  
0  0.338747  0.358964  0.199706     4.166667  
1  0.589347  0.713236 -0.886923     0.000000  
2  0.756269  0.651572  0.423183     4.166667  
3  1.108573  1.153350  1.222550     0.000000  
4  0.066847  0.873676 -0.296106     0.000000  

[5 rows x 29 columns]

<div style="background-color:#EAF4EC; padding:16px; border-radius:10px;">
<h2 style="color:#2F6F4E; margin-bottom:5px; font-size:20px;">
Identification of Temporal Dimensions
</h2>
<p style="color:#2F6F4E; font-size:14px;">
Detection of year-based columns to structure the dataset in a time-aware format suitable for forecasting.
</p>
</div>


In [22]:
# Identify year columns (e.g. 2000, 2001, ..., 2023)
year_cols = [c for c in df_scaled.columns if c.isdigit()]

print(f"Detected {len(year_cols)} year columns:")
print(year_cols[:5], "...", year_cols[-5:])

Detected 24 year columns:
['2000', '2001', '2002', '2003', '2004'] ... ['2019', '2020', '2021', '2022', '2023']


<div style="background-color:#EAF4EC; padding:16px; border-radius:10px;">
<h2 style="color:#2F6F4E; margin-bottom:5px; font-size:20px;">
Reshaping Data to Long Format
</h2>
<p style="color:#2F6F4E; font-size:14px;">
Conversion from wide to long format to obtain a country–indicator–year structure required for time series modeling.
</p>
</div>


In [23]:
# Melt to long format (country–year–indicator)
df_long = df_scaled.melt(
    id_vars=["Country Name", "Country Code", "Indicator Name", "Indicator Code"],
    value_vars=year_cols,
    var_name="year",
    value_name="value"
)

df_long["year"] = df_long["year"].astype(int)

print(df_long.shape)
df_long.head()

(9384, 6)


Country Name Country Code                                    Indicator Name  \
0      Austria          AUT                   Control of Corruption: Estimate   
1      Austria          AUT                             GDP growth (annual %)   
2      Austria          AUT                Government Effectiveness: Estimate   
3      Austria          AUT  Individuals using the Internet (% of population)   
4      Austria          AUT                                     Net migration   

      Indicator Code  year     value  
0             CC.EST  2000  0.986735  
1  NY.GDP.MKTP.KD.ZG  2000  0.177089  
2             GE.EST  2000  1.283242  
3     IT.NET.USER.ZS  2000 -1.284943  
4        SM.POP.NETM  2000 -0.197859

Each line represetn 1 country x 1 indicator x 1 year

In [24]:
df_long.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9384 entries, 0 to 9383
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Country Name    9384 non-null   object 
 1   Country Code    9384 non-null   object 
 2   Indicator Name  9384 non-null   object 
 3   Indicator Code  9384 non-null   object 
 4   year            9384 non-null   int64  
 5   value           9384 non-null   float64
dtypes: float64(1), int64(1), object(4)
memory usage: 440.0+ KB


<div style="background-color:#EAF4EC; padding:16px; border-radius:10px;">
<h2 style="color:#2F6F4E; margin-bottom:5px; font-size:20px;">
Indicator Selection for Forecasting
</h2>
<p style="color:#2F6F4E; font-size:14px;">
Isolation of a single governance indicator to enable focused and interpretable forecasting experiments.
</p>
</div>


In [25]:
# Select one indicator to start with
indicator_name = "Control of Corruption: Estimate"

df_ind = df_long[df_long["Indicator Name"] == indicator_name].copy()

print(df_ind.shape)
df_ind.head()

(648, 6)


Country Name Country Code                   Indicator Name Indicator Code  \
0       Austria          AUT  Control of Corruption: Estimate         CC.EST   
15      Belgium          BEL  Control of Corruption: Estimate         CC.EST   
30     Bulgaria          BGR  Control of Corruption: Estimate         CC.EST   
45      Croatia          HRV  Control of Corruption: Estimate         CC.EST   
59       Cyprus          CYP  Control of Corruption: Estimate         CC.EST   

    year     value  
0   2000  0.986735  
15  2000  0.598961  
30  2000 -1.473102  
45  2000 -1.290505  
59  2000  0.132889

In [26]:
df_ind["Country Name"].nunique(), df_ind["year"].min(), df_ind["year"].max()

(27, np.int64(2000), np.int64(2023))

<div style="background-color:#EAF4EC; padding:16px; border-radius:10px;">
<h2 style="color:#2F6F4E; margin-bottom:5px; font-size:20px;">
Lag Feature Engineering
</h2>
<p style="color:#2F6F4E; font-size:14px;">
Generation of lagged values to capture short-term temporal dependencies within each country.
</p>
</div>


In [27]:
# Create lag features (per country)
for lag in [1, 2, 3]:
    df_ind[f"lag_{lag}"] = (
        df_ind
        .groupby("Country Name")["value"]
        .shift(lag)
    )

df_ind.head(10)

Country Name Country Code                   Indicator Name Indicator Code  \
0        Austria          AUT  Control of Corruption: Estimate         CC.EST   
15       Belgium          BEL  Control of Corruption: Estimate         CC.EST   
30      Bulgaria          BGR  Control of Corruption: Estimate         CC.EST   
45       Croatia          HRV  Control of Corruption: Estimate         CC.EST   
59        Cyprus          CYP  Control of Corruption: Estimate         CC.EST   
73       Czechia          CZE  Control of Corruption: Estimate         CC.EST   
88       Denmark          DNK  Control of Corruption: Estimate         CC.EST   
102      Estonia          EST  Control of Corruption: Estimate         CC.EST   
117      Finland          FIN  Control of Corruption: Estimate         CC.EST   
132       France          FRA  Control of Corruption: Estimate         CC.EST   

     year     value  lag_1  lag_2  lag_3  
0    2000  0.986735    NaN    NaN    NaN  
15   2000  0.598961    NaN    NaN    NaN  
30   2000 -1.473102    NaN    NaN    NaN  
45   2000 -1.290505    NaN    NaN    NaN  
59   2000  0.132889    NaN    NaN    NaN  
73   2000 -1.059243    NaN    NaN    NaN  
88   2000  1.713486    NaN    NaN    NaN  
102  2000 -0.173448    NaN    NaN    NaN  
117  2000  1.773473    NaN    NaN    NaN  
132  2000  0.439446    NaN    NaN    NaN

<div style="background-color:#EAF4EC; padding:16px; border-radius:10px;">
<h2 style="color:#2F6F4E; margin-bottom:5px; font-size:20px;">
Rolling Window Statistics
</h2>
<p style="color:#2F6F4E; font-size:14px;">
Computation of rolling mean and volatility measures to smooth noise and represent medium-term trends.
</p>
</div>


In [28]:
# Garantir ordenação temporal correta
df_ind = df_ind.sort_values(["Country Name", "year"]).copy()

g = df_ind.groupby("Country Name")["value"]

# Rolling statistics (window = 3 years) — apenas informação passada
df_ind["rolling_mean_3"] = g.transform(
    lambda s: s.shift(1).rolling(window=3).mean()
)

df_ind["rolling_std_3"] = g.transform(
    lambda s: s.shift(1).rolling(window=3).std()
)

# Trend (short-term momentum)
df_ind["trend"] = df_ind["lag_1"] - df_ind["lag_2"]

df_ind.head(10)

Country Name Country Code                   Indicator Name  \
0         Austria          AUT  Control of Corruption: Estimate   
391       Austria          AUT  Control of Corruption: Estimate   
782       Austria          AUT  Control of Corruption: Estimate   
1173      Austria          AUT  Control of Corruption: Estimate   
1564      Austria          AUT  Control of Corruption: Estimate   
1955      Austria          AUT  Control of Corruption: Estimate   
2346      Austria          AUT  Control of Corruption: Estimate   
2737      Austria          AUT  Control of Corruption: Estimate   
3128      Austria          AUT  Control of Corruption: Estimate   
3519      Austria          AUT  Control of Corruption: Estimate   

     Indicator Code  year     value     lag_1     lag_2     lag_3  \
0            CC.EST  2000  0.986735       NaN       NaN       NaN   
391          CC.EST  2001  0.986735  0.986735       NaN       NaN   
782          CC.EST  2002  1.195920  0.986735  0.986735       NaN   
1173         CC.EST  2003  1.258012  1.195920  0.986735  0.986735   
1564         CC.EST  2004  1.338009  1.258012  1.195920  0.986735   
1955         CC.EST  2005  1.192433  1.338009  1.258012  1.195920   
2346         CC.EST  2006  1.182813  1.192433  1.338009  1.258012   
2737         CC.EST  2007  1.308334  1.182813  1.192433  1.338009   
3128         CC.EST  2008  1.085782  1.308334  1.182813  1.192433   
3519         CC.EST  2009  0.914044  1.085782  1.308334  1.182813   

      rolling_mean_3  rolling_std_3     trend  
0                NaN            NaN       NaN  
391              NaN            NaN       NaN  
782              NaN            NaN  0.000000  
1173        1.056464       0.120773  0.209185  
1564        1.146889       0.142129  0.062092  
1955        1.263980       0.071232  0.079997  
2346        1.262818       0.072907 -0.145576  
2737        1.237752       0.086959 -0.009620  
3128        1.227860       0.069858  0.125521  
3519        1.192309       0.111580 -0.222553

<div style="background-color:#EAF4EC; padding:16px; border-radius:10px;">
<h2 style="color:#2F6F4E; margin-bottom:5px; font-size:20px;">
Handling Initial Missing Values
</h2>
<p style="font-size:14px; color:#2F6F4E">
Removal of initial observations affected by lag and rolling window construction to ensure model validity.
</p>
</div>


In [29]:
# Drop initial rows with NaN caused by lags/rolling
req = ["value","lag_1","lag_2","lag_3","rolling_mean_3","rolling_std_3","trend"]
df_model = df_ind.dropna(subset=req).reset_index(drop=True)

print(df_model.shape)
df_model.head(100)

(567, 12)


Country Name Country Code                   Indicator Name Indicator Code  \
0       Austria          AUT  Control of Corruption: Estimate         CC.EST   
1       Austria          AUT  Control of Corruption: Estimate         CC.EST   
2       Austria          AUT  Control of Corruption: Estimate         CC.EST   
3       Austria          AUT  Control of Corruption: Estimate         CC.EST   
4       Austria          AUT  Control of Corruption: Estimate         CC.EST   
..          ...          ...                              ...            ...   
95       Cyprus          CYP  Control of Corruption: Estimate         CC.EST   
96       Cyprus          CYP  Control of Corruption: Estimate         CC.EST   
97       Cyprus          CYP  Control of Corruption: Estimate         CC.EST   
98       Cyprus          CYP  Control of Corruption: Estimate         CC.EST   
99       Cyprus          CYP  Control of Corruption: Estimate         CC.EST   

    year     value     lag_1     lag_2     lag_3  rolling_mean_3  \
0   2003  1.258012  1.195920  0.986735  0.986735        1.056464   
1   2004  1.338009  1.258012  1.195920  0.986735        1.146889   
2   2005  1.192433  1.338009  1.258012  1.195920        1.263980   
3   2006  1.182813  1.192433  1.338009  1.258012        1.262818   
4   2007  1.308334  1.182813  1.192433  1.338009        1.237752   
..   ...       ...       ...       ...       ...             ...   
95  2014  0.119426  0.333394  0.339480 -0.150088        0.174262   
96  2015 -0.007413  0.119426  0.333394  0.339480        0.264100   
97  2016 -0.239056 -0.007413  0.119426  0.333394        0.148469   
98  2017 -0.293624 -0.239056 -0.007413  0.119426       -0.042348   
99  2018 -0.470682 -0.293624 -0.239056 -0.007413       -0.180031   

    rolling_std_3     trend  
0        0.120773  0.209185  
1        0.142129  0.062092  
2        0.071232  0.079997  
3        0.072907 -0.145576  
4        0.086959 -0.009620  
..            ...       ...  
95       0.280912 -0.006086  
96       0.125328 -0.213968  
97       0.172249 -0.126839  
98       0.181777 -0.231643  
99       0.151961 -0.054568  

[100 rows x 12 columns]

Initial observations lacking sufficient historical context were removed after temporal feature construction to ensure model validity and prevent information leakage.

<div style="background-color:#EAF4EC; padding:16px; border-radius:10px;">
<h2 style="color:#2F6F4E; margin-bottom:5px; font-size:20px;">
Time-Based Dataset Splitting
</h2>
<p style="color:#2F6F4E; font-size:14px;">
Separation of data into training, validation, and test sets following a strict chronological order.
</p>
</div>


In [30]:
train_end_year = 2018
val_end_year = 2021

# Split using time (NO shuffle, NO random seed)
train_df = df_model[df_model["year"] <= train_end_year].copy()
val_df = df_model[(df_model["year"] > train_end_year) & (df_model["year"] <= val_end_year)].copy()
test_df = df_model[df_model["year"] > val_end_year].copy()

# Basic checks
print("Train:", train_df.shape, "| Years:", train_df["year"].min(), "-", train_df["year"].max())
print("Val:  ", val_df.shape,   "| Years:", val_df["year"].min(), "-", val_df["year"].max())
print("Test: ", test_df.shape,  "| Years:", test_df["year"].min(), "-", test_df["year"].max())

# Check number of countries in each split (should be 27)
print("\nUnique countries:")
print("Train:", train_df["Country Name"].nunique())
print("Val:  ", val_df["Country Name"].nunique())
print("Test: ", test_df["Country Name"].nunique())

print("\nUnique indicators:")
print("Train:", train_df["Indicator Name"].nunique())
print("Val:  ", val_df["Indicator Name"].nunique())
print("Test: ", test_df["Indicator Name"].nunique())

Train: (432, 12) | Years: 2003 - 2018
Val:   (81, 12) | Years: 2019 - 2021
Test:  (54, 12) | Years: 2022 - 2023

Unique countries:
Train: 27
Val:   27
Test:  27

Unique indicators:
Train: 1
Val:   1
Test:  1


Percentage-based splits were avoided due to the temporal nature of the forecasting task.

<div style="background-color:#EAF4EC; padding:16px; border-radius:10px;">
<h2 style="color:#2F6F4E; margin-bottom:5px; font-size:20px;">
Baseline Forecasting Models
</h2>
<p style="color:#2F6F4E; font-size:14px;">
Implementation of simple benchmark models to establish a minimum performance threshold.
</p>
</div>


In [31]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

def eval_baseline(df, y_true_col="value", y_pred_col="y_pred"):
    y_true = df[y_true_col].values
    y_pred = df[y_pred_col].values
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    return mae, rmse


# Baseline 1: Persistence (naive)
# y_hat(t) = y(t-1)
val_naive = val_df.copy()
test_naive = test_df.copy()

val_naive["y_pred"] = val_naive["lag_1"]
test_naive["y_pred"] = test_naive["lag_1"]

mae_val_naive, rmse_val_naive = eval_baseline(val_naive)
mae_test_naive, rmse_test_naive = eval_baseline(test_naive)

print("Naive")
print(f"Validation -> MAE: {mae_val_naive:.4f} | RMSE: {rmse_val_naive:.4f}")
print(f"Test       -> MAE: {mae_test_naive:.4f} | RMSE: {rmse_test_naive:.4f}")

# Baseline 2: Moving Average (window=3)
# y_hat(t) = mean(y(t-1), y(t-2), y(t-3))

val_ma = val_df.copy()
test_ma = test_df.copy()

# Option A: use rolling_mean_3 (computed on value with window=3)
# This corresponds to mean of the last 3 observed values at time t
val_ma["y_pred"]  = (val_ma["lag_1"] + val_ma["lag_2"] + val_ma["lag_3"]) / 3
test_ma["y_pred"] = (test_ma["lag_1"] + test_ma["lag_2"] + test_ma["lag_3"]) / 3

mae_val_ma, rmse_val_ma = eval_baseline(val_ma)
mae_test_ma, rmse_test_ma = eval_baseline(test_ma)

print("\nMoving Average (3-year)")
print(f"Validation -> MAE: {mae_val_ma:.4f} | RMSE: {rmse_val_ma:.4f}")
print(f"Test       -> MAE: {mae_test_ma:.4f} | RMSE: {rmse_test_ma:.4f}")

Naive
Validation -> MAE: 0.0963 | RMSE: 0.1332
Test       -> MAE: 0.0729 | RMSE: 0.0890

Moving Average (3-year)
Validation -> MAE: 0.1242 | RMSE: 0.1640
Test       -> MAE: 0.0812 | RMSE: 0.1061


Baseline results confirm the strong temporal persistence of governance indicators. The naïve persistence model outperforms the moving average benchmark, indicating that simple temporal smoothing does not improve predictive accuracy. As a result, the naïve model is adopted as the primary baseline against which all advanced forecasting models are evaluated.